In [42]:
from sklearnex import patch_sklearn
patch_sklearn()

import pandas as pd
import seaborn as sns
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
import time
from sklearn.preprocessing import LabelEncoder
import optuna
from sklearn.model_selection import cross_val_score

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [43]:
train = pd.read_csv("train_all_with_seasonal_cluster.csv")
test = pd.read_csv("test_all_with_seasonal_cluster.csv")

In [44]:
train.head()

,ID,cluster_id,electricity_consumption,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,month,dayofweek,quarter,day,weekofyear
0,cluster_1_2014-01-01,0,358.032,10.8,4.2,5.5,0.4,53.003333,29787.533333,40.6,79.2,186.000000,1.176667,0.483333,1,2,1,1,1
1,cluster_2_2014-01-01,1,548.247,12.2,4.3,8.6,-0.4,8195.656667,30650.350000,36.0,81.7,170.591118,3.383333,0.736667,1,2,1,1,1
2,cluster_3_2014-01-01,2,758.303,12.9,-0.8,10.1,-4.9,16305.260000,31547.686667,20.9,44.3,159.467752,3.880000,0.716667,1,2,1,1,1
3,cluster_4_2014-01-01,3,1072.077,10.8,4.7,6.7,0.6,9224.803333,30769.220000,34.7,82.8,184.339753,3.153333,0.676667,1,2,1,1,1
4,cluster_1_2014-01-02,0,386.908,10.7,7.0,6.6,3.3,22372.000000,29850.226667,34.3,75.2,210.333465,3.640000,0.780000,1,3,1,2,1


In [45]:
target_col = "electricity_consumption"

In [ ]:
def train_and_eval_optuna(df_input, label, n_trials=50):
    X = df_input.drop(columns=[target_col])
    y = df_input[target_col]
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)

    def objective(trial):
        params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'n_estimators': trial.suggest_categorical('n_estimators', [100, 300, 500, 700, 1000]),
            'max_depth': trial.suggest_categorical('max_depth', [3, 5, 7, 9, 11]),
            'learning_rate': trial.suggest_categorical('learning_rate', [0.005, 0.01, 0.05, 0.1, 0.2]),
            'subsample': trial.suggest_categorical('subsample', [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
            'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
            'gamma': trial.suggest_categorical('gamma', [0, 0.1, 0.25, 0.5, 1, 5]),
            'min_child_weight': trial.suggest_categorical('min_child_weight', [1, 2, 3, 5, 7, 10]),
            'reg_alpha': trial.suggest_categorical('reg_alpha', [0, 0.001, 0.01, 0.1, 1]),
            'reg_lambda': trial.suggest_categorical('reg_lambda', [0.1, 0.5, 1.0, 2.0, 5.0]),
        }

        evals = [(dval, 'eval')]
        booster = xgb.train(params, dtrain, num_boost_round=1000, evals=evals, early_stopping_rounds=20, verbose_eval=False)

        preds = booster.predict(dval)
        rmse = mean_squared_error(y_val, preds, squared=False)
        return rmse

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params
    best_params.update({
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
    })

    final_model = xgb.train(
        best_params,
        dtrain,
        num_boost_round=1000,
        evals=[(dval, 'eval')],
        early_stopping_rounds=20,
        verbose_eval=False
    )

    y_pred = final_model.predict(dval)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    print(f"Optuna-tuned {label} RMSE: {rmse:.4f}")
    print("Best params:", best_params)

    return final_model, y_pred

In [47]:
train_cluster_date = train.drop(columns=["ID"])

In [48]:
best_model, predictions = train_and_eval_optuna(train_cluster_date, label="ElectricityConsumption", n_trials=50)

[I 2025-06-27 10:24:57,641] A new study created in memory with name: no-name-3c560f57-ed7a-4ba6-b2b3-94c8260700eb
c:\Users\HP\anaconda3\Lib\site-packages\xgboost\callback.py:386: UserWarning: [10:24:57] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "n_estimators" } are not used.

  self.starting_round = model.num_boosted_rounds()
c:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2025-06-27 10:25:00,109] Trial 0 finished with value: 28.88091954730121 and parameters: {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.1, 'subsample': 1.0, 'colsample_bytree': 0.8, 'gamma': 0.5, 'min_child_weight': 1, 'reg_alpha': 0.1, 'reg_lambda': 1.0}. Best is trial 0 with value: 28.88091954730121.
c:\Users\HP\anaconda3\Lib\site-packages\xgboost\callba

Optuna-tuned ElectricityConsumption RMSE: 26.9796
Best params: {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.05, 'subsample': 0.9, 'colsample_bytree': 0.6, 'gamma': 0, 'min_child_weight': 3, 'reg_alpha': 0, 'reg_lambda': 0.5, 'objective': 'reg:squarederror', 'eval_metric': 'rmse'}


c:\Users\HP\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [51]:
X_test = test.drop(columns=["ID"]) 

In [54]:
dtest = xgb.DMatrix(X_test)
y_test_pred = best_model.predict(dtest)

In [55]:
submission = pd.DataFrame({
    "ID": test["ID"],
    "electricity_consumption": y_test_pred
})
submission.to_csv("submission16.csv", index=False)